In [ ]:
import os, re
from pathlib import Path

In [ ]:
def process(addr_start, addr_end, path, prefix, output):
    path = Path(path)
    for f in os.listdir(path):
        if "-maps.txt" in f:
            continue
        name = f[len(prefix):]
        start = int(name[:name.find("-")], base=16)
        end = int(name[name.find("-")+1:name.find(".bin")], base=16)
        if start <= addr_start and end >= addr_end:
            break
    print(f, hex(start), hex(end))
    with open(path / f, "rb") as fp:
        data = fp.read()
    data = data[addr_start-start:len(data)-(end-addr_end)]
    with open(path / output, "wb") as fp:
        fp.write(data)

In [ ]:
def autoProcess(path, target="libil2cpp.so"):
    path = Path(path)
    suffix = "-maps.txt"
    maps_file = [f for f in os.listdir(path) if f.endswith(suffix)][0]
    prefix = maps_file[:-len(suffix)+1]
    with open(path / maps_file, "r") as f:
        lines = f.readlines()
    start = None
    end = None
    re_addr = re.compile("^([a-f0-9]+)-([a-f0-9]+)")
    for line in lines:
        if not target in line:
            continue
        m = re_addr.match(line)
        st, ed = [int(v, base=16) for v in m.groups()]
        if start is None:
            start = st
        end = ed
    print(f"target {hex(start)}-{hex(end)}")
    process(start, end, path, prefix, target)

In [ ]:
autoProcess("G:/Downloads/Dump")